# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [1]:
# %pip install --upgrade tensorflow[and-cuda]
# %pip install -q huggingface_hub

In [2]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import huggingface_hub
import os
lead_id="AIBridgeEngine/Ben-3B-Lead-v0.1"
bidding_id="AIBridgeEngine/Ben-3B-Bidding-v0.1"
biddinginfo_id="AIBridgeEngine/Ben-3B-Biddinginformation-v0.1"
singledummy_id="AIBridgeEngine/Ben-3B-Singledummy-v0.1"
folder="hfmodels"
huggingface_hub.snapshot_download(repo_id=lead_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=bidding_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=biddinginfo_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=singledummy_id, local_dir=folder)

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 14.71it/s]


'D:\\github\\ben\\src\\hfmodels'

In [4]:
from nn.leader import Leader
from nn.bid_info import BidInfo
from nn.bidder import Bidder
from nn.lead_singledummy import LeadSingleDummy
lead_suit = Leader(f'{folder}/lead-1000000')
lead_nt = Leader(f'{folder}/lead-1000000')
biddinginfo = BidInfo(f'{folder}/binfo-100000')
bidder = Bidder("", f'{folder}/bidding-100000')
sd_model = LeadSingleDummy(f'{folder}/lr3-1000000')
models = Models(bidder, biddinginfo, lead_suit, lead_nt, sd_model, None, None, 0.1, 0.05, 1.0, 0.1, False, -1, -1, False, True, False, False, 1)
sampler = Sample(0.01, 0.1, 0.03, 0.05, 64, 10, 5000, 5000, 128, 100, 10, 500, False)

INFO:tensorflow:Restoring parameters from hfmodels/lead-1000000
INFO:tensorflow:Restoring parameters from hfmodels/lead-1000000
INFO:tensorflow:Restoring parameters from hfmodels/binfo-100000
INFO:tensorflow:Restoring parameters from hfmodels/bidding-100000
INFO:tensorflow:Restoring parameters from hfmodels/lr3-1000000


In [5]:
# both vulnerable. you are sitting North as dealer and you hold
hand = 'T54.QT87.A98.943'

# the auction goes:
auction = ["1C","1D","1S","2D","X","PASS","2S","PASS","3C","PASS","3N","PASS","4S",'PASS', 'PASS', 'PASS']

# what to lead?

lead_bot = BotLead([True, True], hand, models, sampler, False)
lead = lead_bot.find_opening_lead(auction)

In [6]:
lead.card

H7

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [7]:
lead.to_dict()['candidates']

[{'card': 'Hx',
  'insta_score': 0.612,
  'expected_tricks_sd': 9.34,
  'p_make_contract': 0.51},
 {'card': 'DA',
  'insta_score': 0.159,
  'expected_tricks_sd': 9.38,
  'p_make_contract': 0.51},
 {'card': 'HT',
  'insta_score': 0.065,
  'expected_tricks_sd': 9.35,
  'p_make_contract': 0.51}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [8]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['KQx.AJx..KQ8xxxx AJx.Kxx.KJxxx.JT 98xx.9xx.QTxxx.A 0.09477',
 'Axx.AK9x.x.AQxxx K9x.xx.QJxxx.KJ8 QJ8x.Jxx.KTxx.Tx 0.07342',
 'KJx.AJ9x.K.KJ8xx A8x.xxx.JTxxx.Ax Q9xx.Kx.Qxxx.QTx 0.06904',
 'AKQ.AJxx..QT8xxx 8xx.xx.KQxxxx.AJ J9xx.K9x.JTxx.Kx 0.06505',
 'Qxx.AJ9x.K.AQJ8x K9x.xx.QTxxx.KTx AJ8x.Kxx.Jxxx.xx 0.05431',
 'J9x.AKJx.x.AQ8xx A8x.9xx.QJxxx.Jx KQxx.xx.KTxx.KTx 0.05310',
 'A9x.KJ9x.x.AK8xx Kxx.Ax.QTxxx.JTx QJ8x.xxx.KJxx.Qx 0.05246',
 'Kx.K9xx.xx.AKQxx AJx.Jx.QTxxx.J8x Q98xx.Axx.KJx.Tx 0.04814',
 'AJ8.xxx..AKQ8xxx K9x.AKJx.Jxxxx.x Qxxx.9x.KQTxx.JT 0.04288',
 'K9x.Jx.x.AKQJ8xx Ax.K9xx.QJTxxx.x QJ8xx.Axx.Kxx.Tx 0.03857']